In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qqqqqq/gibVirus.fa


In [2]:
!git clone https://github.com/rosericazondekon/VirusSeqPipeline.git

Cloning into 'VirusSeqPipeline'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), 7.01 MiB | 4.73 MiB/s, done.


In [3]:
ls -l /kaggle/working/VirusSeqPipeline/Mosaik_JumpDb/

total 8632
-rw-r--r-- 1 root root    1175 Oct 28 19:15 Jump_file_builder.sh
-rw-r--r-- 1 root root     990 Oct 28 19:15 Spanner_anchor_hg19Virus.txt
-rw-r--r-- 1 root root 8828983 Oct 28 19:15 hg19Virus_refGene_RIS.txt


In [4]:
!chmod 666 /kaggle/working/VirusSeqPipeline/Mosaik_JumpDb/*

In [5]:
!if [ ! -f ./hg19.fa ] ; then
!echo "Downloading hg19.fa.gz..."
!wget "http://odin.mdacc.tmc.edu/~xsu1/hg19.fa.gz"
!echo "Extracting hg19.fa.gz"
!gunzip hg19.fa.gz
!cp /kaggle/input/qqqqqq/gibVirus.fa .
!fi

/bin/bash: -c: line 1: syntax error: unexpected end of file
--2023-10-28 19:15:09--  http://odin.mdacc.tmc.edu/~xsu1/hg19.fa.gz
Resolving odin.mdacc.tmc.edu (odin.mdacc.tmc.edu)... 143.111.2.220
Connecting to odin.mdacc.tmc.edu (odin.mdacc.tmc.edu)|143.111.2.220|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://odin.mdacc.tmc.edu/~xsu1/hg19.fa.gz [following]
--2023-10-28 19:15:09--  https://odin.mdacc.tmc.edu/~xsu1/hg19.fa.gz
Connecting to odin.mdacc.tmc.edu (odin.mdacc.tmc.edu)|143.111.2.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 944601745 (901M) [application/x-gzip]
Saving to: ‘hg19.fa.gz’

hg19.fa.gz          100%[===================>] 900.84M  1.05MB/s    in 14m 32s 

2023-10-28 19:29:41 (1.03 MB/s) - ‘hg19.fa.gz’ saved [944601745/944601745]

Extracting hg19.fa.gz
/bin/bash: -c: line 0: syntax error near unexpected token `fi'
/bin/bash: -c: line 0: `fi'


In [6]:
!wget http://downloads.hmpdacc.org/data/Illumina/anterior_nares/SRS054061.tar.bz2

--2023-10-28 19:30:06--  http://downloads.hmpdacc.org/data/Illumina/anterior_nares/SRS054061.tar.bz2
Resolving downloads.hmpdacc.org (downloads.hmpdacc.org)... 134.192.156.26
Connecting to downloads.hmpdacc.org (downloads.hmpdacc.org)|134.192.156.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138711242 (132M) [application/x-bzip2]
Saving to: ‘SRS054061.tar.bz2’

SRS054061.tar.bz2   100%[===================>] 132.29M  28.0MB/s    in 6.1s    

2023-10-28 19:30:13 (21.5 MB/s) - ‘SRS054061.tar.bz2’ saved [138711242/138711242]



In [7]:
!tar -xvjf /kaggle/working/SRS054061.tar.bz2 -C /kaggle/working/VirusSeqPipeline/samples

tar: /kaggle/working/VirusSeqPipeline/samples: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [8]:
!rm hg19.fa

In [9]:
!/kaggle/working/VirusSeqPipeline/Mosaik_bin/MosaikBuild -fr hg19.fa -oa hg19.fa.bin -st illumina

/bin/bash: /kaggle/working/VirusSeqPipeline/Mosaik_bin/MosaikBuild: Permission denied


In [10]:
!/kaggle/working/VirusSeqPipeline/Mosaik_bin/MosaikJump -ia hg19.fa.bin -out hg19.JumpDb -hs 6 -mhp 100 -km

/bin/bash: /kaggle/working/VirusSeqPipeline/Mosaik_bin/MosaikJump: Permission denied


In [11]:
!mv *.jmp /kaggle/working/VirusSeqPipeline/Mosaik_JumpDb/
!mv *.bin /kaggle/working/VirusSeqPipeline/Mosaik_JumpDb/

mv: cannot stat '*.jmp': No such file or directory
mv: cannot stat '*.bin': No such file or directory


In [12]:
cp /kaggle/input/qqqqqq/gibVirus.fa /kaggle/working/

In [13]:
!mkdir /kaggle/working/VirusSeqPipeline/samples

In [14]:
#!/usr/bin/env python
import os
import subprocess

# Set the working directory where the script will run
working_directory = "/kaggle/working"
os.chdir(working_directory)

# VirusSeqPipeline by Roseric Azondekon

# Set the VirusSeq working directory
folder = "/kaggle/working/VirusSeqPipeline"

# Get Reference genome files if they do not exist

# Check if all Jump files exist. If not, create them
'''jump_files = os.listdir(os.path.join(folder, "Mosaik_JumpDb"))
if len(jump_files) != 6:
    os.chdir(os.path.join(folder, "Mosaik_JumpDb"))
    subprocess.call(["bash", "Jump_file_builder.sh"])
else:
    print("All JMP files already exist...Skip Jump Db creation...")'''

# For each sample in the folder samples
folder2 = os.path.join(folder, "samples")
sample_folders = os.listdir(folder2)
for sample_folder in sample_folders:
    sample = sample_folder  # Use the folder name as the sample name

    # All job processing goes here

    # Create folders if they do not exist
    os.makedirs(os.path.join(folder2, sample_folder, "Gig"), exist_ok=True)
    os.makedirs(os.path.join(folder2, sample_folder, "SV_gDNA"), exist_ok=True)
    os.makedirs(os.path.join(folder, "results", sample), exist_ok=True)

    # Virus detection by NGS data

    # Converting external read formats to the native MOSAIK format
    subprocess.call([
        os.path.join(folder, "Mosaik_bin", "MosaikBuild"),
        "-q", os.path.join(folder2, sample_folder, f"{sample}.denovo_duplicates_marked.trimmed.1.fastq"),
        "-q2", os.path.join(folder2, sample_folder, f"{sample}.denovo_duplicates_marked.trimmed.2.fastq"),
        "-out", os.path.join(folder, "results", sample, f"{sample}_Virus.bin"),
        "-st", "illumina"
    ])

    # Performing alignment against the human genome reference hg19 with MosaikAligner
    subprocess.call([
        os.path.join(folder, "Mosaik_bin", "MosaikAligner"),
        "-in", os.path.join(folder, "results", sample, f"{sample}_Virus.bin"),
        "-ia", os.path.join(folder, "Mosaik_JumpDb", "hg19.fa.bin"),
        "-out", os.path.join(folder, "results", sample, f"{sample}_Virus.bin.aligned"),
        "-hs", "6",
        "-mmp", "0.1",
        "-mmal", "0.5",
        "-act", "25",
        "-mhp", "100",
        "-m", "unique",
        "-j", os.path.join(folder, "Mosaik_JumpDb", "hg19.JumpDb"),
        "-p", "14",
        "-km",
        "-pm",
        "-rur", os.path.join(folder, "results", sample, f"{sample}_Unalg.fq")
    ])
    
    # Continue with the rest of the script...
    # Start aligning unmapped reads against virus genomes to detect the virus

    # Sorting the mapped reads in terms of their mapped genomic location using MosaikSort
    subprocess.call([
        os.path.join(folder, "Mosaik_bin", "MosaikBuild"),
        "-q", os.path.join(folder, "results", sample, f"{sample}_Unalg.fq"),
        "-out", os.path.join(folder, "results", sample, f"{sample}_Virus.bin"),
        "-st", "illumina"
    ])

    subprocess.call([
        os.path.join(folder, "Mosaik_bin", "MosaikAligner"),
        "-in", os.path.join(folder, "results", sample, f"{sample}_Virus.bin"),
        "-ia", os.path.join(folder, "Mosaik_JumpDb", "gibVirus.fa.bin"),
        "-out", os.path.join(folder, "results", sample, f"{sample}_Virus.bin.aligned"),
        "-hs", "6",
        "-mmp", "0.15",
        "-act", "25",
        "-mhp", "100",
        "-m", "all",
        "-j", os.path.join(folder, "Mosaik_JumpDb", "gibVirus.JumpDb"),
        "-p", "14",
        "-km",
        "-pm"
    ])

    subprocess.call([
        os.path.join(folder, "Mosaik_bin", "MosaikSort"),
        "-in", os.path.join(folder, "results", sample, f"{sample}_Virus.bin.aligned"),
        "-out", os.path.join(folder, "results", sample, f"{sample}_Virus.bin.aligned.sorted")
    ])
    '''
    # Producing an assembly of the reads pileup for visualization purposes using MosaikAssembler
    subprocess.call([
        os.path.join(folder, "Mosaik_bin", "MosaikAssembler"),
        "-in", os.path.join(folder, "results", sample, f"{sample}_Virus.bin.aligned.sorted"),
        "-ia", os.path.join(folder, "Mosaik_JumpDb", "gibVirus.fa.bin"),
        "-out", os.path.join(folder2, sample_folder, "Gig", f"{sample}_Virus.bin.aligned.sorted.assembled"),
        "-f", "ace"
    ])
    '''
    mosaik_assembler_command = [
    os.path.join(folder, "Mosaik_bin", "MosaikAssembler"),
    "-in", os.path.join(folder, "results", sample, f"{sample}_Virus.bin.aligned.sorted"),
    "-ia", os.path.join(folder, "Mosaik_JumpDb", "gibVirus.fa.bin"),
    "-out", os.path.join(folder2, sample_folder, "Gig", f"{sample}_Virus.bin.aligned.sorted.assembled"),
    "-f", "ace"
    ]

    # Specify the output file
    virus_log_output = open(os.path.join(folder, "results", sample, f"{sample}_VirusLog.txt"), "w")

    # Run MosaikAssembler and redirect output to VirusLog.txt
    subprocess.call(mosaik_assembler_command, stdout=virus_log_output)
    virus_log_output.close()  # Close the output file
    
    # Detect the virus in the sample by VirusSeq_Detection.pl
    subprocess.call([
        "perl",
        os.path.join(folder, "VirusSeq_Script", "VirusSeq_Detection.pl"),
        os.path.join(folder, "results", sample, f"{sample}_VirusLog.txt"),
        "10",
        os.path.join(folder, "results", sample, f"{sample}_VirusName.txt")
    ])
    
    # Continue with the next sample

# The rest of your script here...

# Detection of virus integration sites by NGS data

# Continue with the script to process other samples and tasks

# End of the script

In [15]:
!rm *.tar.bz2
!rm -rf /kaggle/working/VirusSeqPipeline/samples/*

In [16]:
from IPython.display import FileLink
# Compress the files in the specified directory
!tar -czvf /kaggle/working/results.zip -C /kaggle/working/VirusSeqPipeline/results .
# Create a FileLink to the compressed file
FileLink(r'/kaggle/working/results.zip')

tar: /kaggle/working/VirusSeqPipeline/results: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


/kaggle/working/results.zip